# Order a Basemap Using the Orders API & SDK via AOI & Quad ID

## Overview ##
---
In this notebook, you will learn how to order a [Planet Basemap](https://developers.planet.com/docs/data/visual-basemaps/) using your [Area of Interest](https://developers.planet.com/apis/orders/basemaps/#order-basemaps-by-area-of-interest-aoi) (AOI) and a [Quad ID](https://developers.planet.com/apis/orders/basemaps/#order-basemaps-by-quad-ids-and-deliver-to-cloud). We will place this order via Planet's [Orders API](https://developers.planet.com/apis/orders/) using our Planet Python [SDK](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/). 

1. Get a Basemap ID using either [Planet Explorer](https://developers.planet.com/docs/apps/explorer/) or the [Basemap Viewer](https://www.planet.com/basemaps/#/mosaic/). 
2. Create a JSON order packet with order parameters.
3. Set up a session with the Planet SDK, and place the order, [delivering]((https://developers.planet.com/apis/orders/delivery/)) the resulting Basemap to a Google Cloud Storage bucket. 
4. Repeat steps 1-3 using quad IDs. 

## Order a Basemap using an AOI
---
### First, find a Basemap to order
You can get a Basemap ID from Planet Explorer or the Basemap Viewer.

### Get a basemap ID from Planet Explorer
You'll need a basemap ID to identify what basemap to download.

To pick a basemap, you can open [Planet Explorer](https://www.planet.com/explorer/#) (you can learn more about Planet Explorer [here](https://developers.planet.com/docs/apps/explorer/)) and select a basemap from the dropdown menu in the lower left corner. (If you don't have access to basemaps, you'll see a message confirming in the lower right corner. Select "Get Access" to sign up for Basemaps or speak to your customer service manager.)

Once you've selected a Basemap, the Explorer updates to include that basemap in the view.

Select the search icon and time cadence, for example monthly or quarterly. Then select one of the resulting basemap IDs, such as `global_monthly_2022_01_mosaic`. That's the value you'll be passing into the Orders API. 

### Get a basemap ID from the Basemap Viewer

To pick a basemap, you can open the [Basemap Viewer](https://www.planet.com/basemaps/#/mosaic/) (you can learn more about the Basemap Viewer [here](https://developers.planet.com/docs/apps/basemapsviewer/)) and select a basemap from the left sidebar menu, using the filter to narrow down the basemap you want. 

Once you've selected a basemap, and selected the right arrow >, the basemap ID displays, such as `global_monthly_2022_01_mosaic`. That's the value you'll be passing into the Orders API.

### Import Planet and Related Packages

---

Make sure you have Planet's Python SDK properly downloaded. You can find out more about this [here](https://developers.planet.com/docs/pythonclient/). Find your [API key](https://developers.planet.com/quickstart/apis/).

Next set up a session by importing needed Python packages, pulling in your API Key, and make an initial request (to retrieve the Orders API parameters) to confirm a connection with the server.

Create a Google Cloud Platform (GCP) storage bucket with these [instructions](https://cloud.google.com/storage/docs/creating-buckets). This is where we are delivering the data. Creating the bucket will create the credentials for you, and you can convert it to Base64 using these [instructions](https://cloud.google.com/vision/docs/base64) and save this as an environment variable. 

In [1]:
import planet
import os

In [2]:
PL_API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
auth = planet.Auth.from_key(PL_API_KEY)

### Create an order packet
---
Package up the details of your order in a [JSON object](https://developers.planet.com/apis/orders/basemaps/#example-order-query-json-block) and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. We are [delivering](https://developers.planet.com/apis/orders/delivery/) this order to a Google Cloud Storage bucket. In this request, we are also using the tools [merge](https://developers.planet.com/apis/orders/tools/#merge) and [clip](https://developers.planet.com/apis/orders/tools/#clip). 

In [3]:
order_params = {
    "name": "basemap order with geometry",
    "source_type": "basemaps",
    "order_type": "partial",
    "products": [
        {
            "mosaic_name": "global_monthly_2022_01_mosaic",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                   [
                       [4.607406, 52.353994],
                       [4.680005, 52.353994],
                       [4.680005, 52.395523],
                       [4.607406, 52.395523],
                       [4.607406, 52.353994]
                   ]
                ]
            }
        }
    ],
    "tools": [
        {"merge": {}},
        {"clip": {}}
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "devrel-notebooks",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/"
        }
    }
}

### Create a session with SDK and poll for success

Here, we are creating an order using the SDK V2. Then we are waiting for the order to be delievered to our GCP storage bucket. Note that this can take some time.

In [4]:
async def create_and_deliver_order(order_params, client):
    '''Create an order and wait for it to delivered

    Parameters:
        order_params: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_params)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'], callback=reporter.update_state,
                          max_attempts=0)

In [5]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GCP
    await create_and_deliver_order(order_params, client)

04:05 - order e7ad9813-ff41-42c3-a4a0-6e13f44fbe7b - state: success


### Check your results

After a few simple steps, we have a basemap delivered to our Google Cloud Platform bucket using the SDK v2.

## Use Basemaps API to get the Basemap and quad IDs
---

### What is a Quad ID? 

A quad ID is a unique identifer to access a square tile of imagery of the Earth's surface, whereas a Basemap is a combination of squares. 

Use the Basemap APIs to retrieve the basemap you want and the quad IDs you are looking for. You can get the base URL you'll need to communicate with the Basemap API service:

1. Go to the [Basemap API reference](https://developers.planet.com/docs/basemaps/reference/) at https://developers.planet.com/docs/basemaps/reference/. 
2. Under List Mosaics, select GET /mosaics.

A dropdown UI element appears with the URL to use:

https://api.planet.com/basemaps/v1/mosaics


In [6]:
import requests
from requests.auth import HTTPBasicAuth
import json
import math

In [7]:
auth = HTTPBasicAuth(PL_API_KEY, '')
session = requests.Session()

# Authenticate
session.auth = (PL_API_KEY, '')

#### Search for the mosaic ID to get its quad IDs

Here, we are using the full name of the mosaic, but with the `name__contains` field, you can use a partial name based on the names of the mosaics available through your account.

Declare the search string for `requests.get()`. You will use the dictionary you create with the mosaic name to pass to the requests.get() function.


This code sets the URL and headers for an HTTP request to Planet's Basemap API and sends a GET request with the specified search parameters. If the response returns the correct status, the JSON response is loaded into a Python dictionary with mosaics information from which we'll extract quad ID information.

In [8]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'

basemap_params = {
    'name__contains': 'point_reyes_rolling_normalized_2020-04-30_mosaic'
}

session.headers.update({'content-type': 'application/json'})

basemap_service_response = session.get(
    url=BASEMAP_API_URL, params=basemap_params)

if basemap_service_response.status_code < 400:
    basemaps = basemap_service_response.json()
else:
    basemap_service_response.raise_for_status()

This code retrieves the ID and bounding box coordinates of the mosaic we loaded above and uses these to construct an API request to retrieve the corresponding quad IDs for a specific area.

In [12]:
mosaic_id = basemaps['mosaics'][0]['id']
mosaic_box = basemaps['mosaics'][0]['bbox']
mosaic_box_str = ','.join(map(str, mosaic_box))

quad_params = {
    'mosaic_id': mosaic_id,
    'bbox': mosaic_box_str,
}

quads_url = "{}/{}/quads".format(BASEMAP_API_URL, mosaic_id)
headers = ({'content-type': 'application/json'})
quad_service_response = session.get(
    url=quads_url, params=quad_params, auth=auth, headers=headers)
quads = quad_service_response.json()
items = quads['items']

quad_ids = [i['id'] for i in items]

### Create an order packet
---
Create an order packet with details including the mosaic name and quad IDs, and the tools and delivery method. In this example, we are using the tools [bandmath](https://developers.planet.com/apis/orders/tools/#band-math) and [reproject](https://developers.planet.com/apis/orders/tools/#reproject). 

#### Bandmath 

The bandmath tool allows you to apply band math expressions to the bands of your input files to produce derived outputs and indices for analysis. Note that `b5` in this example is being assigned to [NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index). 

#### Reproject

Reprojecting a basemap is necessary when the original map projection does not accurately represent the Earth's surface in a specific region or area. This can happen when the imagery for the basemap comes from a place far from the equator. In the example shown in this notebook, there is not a significant change in circumference of the earth from the equator, but it is always best to check using the code below. 

First, find the latitude of the area of interest. Then, find the circumference of the Earth at the given latitude in kilometers. We recommend using a website like [this](http://www.easysurf.cc/circle.htm#cetol1) Use the 'Calculate Circumference of the Earth at a given Latitude' calculator. Enter the latitude of the area you are looking for. In the code below, replace the parameters with your latitude and the circumference of the earth in kilometers. 

In [13]:
def calculate_resolution(latitude):
    '''Calculate the resolution of a basemap at a given latitude

    Parameters: latitude (float): latitude at location of interest [degrees]
    Return (float): resolution of basemap at given latitude [degrees per pixel]
    '''
    earth_radius = 6378137.0  # meters
    native_resolution = 3  # meters per pixel
    circumference_of_earth = 2.0 * math.pi * (earth_radius)
    meters_per_degree_at_latitude = math.cos(math.radians(
        latitude)) * circumference_of_earth / 360.0
    resolution_at_latitude = native_resolution / meters_per_degree_at_latitude
    return resolution_at_latitude

In [14]:
resolution_at_latitude = calculate_resolution(37.0)

In [15]:
order_params = {
    "name": "basemap order with quad_ids",
    "source_type": "basemaps",
    "products": [
        {
            "mosaic_name": "point_reyes_rolling_normalized_2020-04-30_mosaic",
            "quad_ids": quad_ids
        }
    ],
    "tools": [
        {
            "reproject": {
                "projection": "EPSG:4326",
                "resolution": resolution_at_latitude,
                "kernel": "cubic"
            }
        },
        {
            "bandmath": {
                "b1": "b1",
                "b2": "b2",
                "b3": "b3",
                "b4": "b4",
                "b5": "(b4-b3)/(b4+b3)",
                "pixel_type": "32R"
            }
        }
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "devrel-notebooks",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/",
        }
    }
}

### Create and deliver order with the SDK



In [16]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GCP
    await create_and_deliver_order(order_params, client)

13:51 - order bc49e207-814d-4d00-904a-d4d0bc64b278 - state: running

### Check your results

After a few simple steps, we have a basemap delivered to our Google Cloud Platform bucket using the SDK v2.  For more information, check out the SDK v2 docs [here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/). 